In [1]:
import spacy
from spacy.tokens import DocBin
from spacy import displacy
from spacy.util import filter_spans
import pickle
import random
import re
from rich import print as prt

In [2]:
# for i in utils.v2Tov3Converter(train_data):
#     for j in i.ents:
#         prt(j.text, " ", j.label_)
    

In [3]:
train_data = pickle.load(open('train_data/train_data.pkl', 'rb'))

In [10]:
class utils:
    # convert tuples to list and return converted data
    def prepareData(train_data):
        data = []
        for text, annot in train_data:
            ent = []
            for strt, end, lbl in annot['entities']:
                ent.append([strt, end, lbl])
            annot['entities'] = ent
            data.append([text, annot])
        return data

    # visualise training data using displacy
    def renderData(data, start=0, end=1, serve=False):
        nlp = spacy.blank('en')
        data0 = data[start:end]
        for text, annotations in data0:
            doc = nlp.make_doc(text)
            ents = []
            for start, end, label in annotations['entities']:
                span = doc.char_span(start, end, label=label)
                if(span!=None):
                    ents.append(span)
            doc.ents = ents
        if serve:
            displacy.serve(doc, style='ent')
        else:
            displacy.render(doc, style='ent')

    def remove_whitespace_entities(doc):
        doc.ents = [e for e in doc.ents if not e.text.isspace()]
        return doc

    # convert training dataset from v2 to v3 using docbin
    # note use filter_span to to get rid of the span errors
    def v2Tov3Converter(data, filename="train"):
        nlp = spacy.blank("en")
        # the DocBin will store the example documents
        db = DocBin()
        for text, annotations in data:
            doc = nlp.make_doc(text)
            ents = []
            for start, end, label in annotations['entities']:
                # span = doc.char_span(start, end, label=label, alignment_mode='contract')
                span = doc.char_span(start, end, label=label)
                if span == None:
                    continue
                if span.text.isspace()==True:
                    continue
                proceed = True
                span_text = span.text
                if span_text[0]==' ' or span_text[len(span_text)-1]==' ':
                    continue
                for char in span.text:
                    if char.isalnum():
                        continue
                    else:
                        proceed = False

                if proceed:
                    ents.append(span)

            ents = filter_spans(ents)
            # prt(ents)
            doc.ents = ents
            doc = utils.remove_whitespace_entities(doc)
            db.add(doc)
        filename=filename+".spacy"
        db.to_disk(filename)
        return list(db.get_docs(nlp.vocab))
    
    def trim_entity_spans(data: list) -> list:
        # Removes leading and trailing white spaces from entity spans.

        # Args:
        # data (list): The data to be cleaned in spaCy JSON format.

        # Returns:
        # list: The cleaned data.
        invalid_span_tokens = re.compile(r'\s')
        cleaned_data = []
        for text, annotations in data:
            entities = annotations['entities']
            valid_entities = []
            for start, end, label in entities:
                valid_start = start
                valid_end = end
                # if there's preceding spaces, move the start position to nearest character
                while valid_start < len(text) and invalid_span_tokens.match(
                        text[valid_start]):
                    valid_start += 1
                while valid_end > 1 and invalid_span_tokens.match(
                        text[valid_end - 1]):
                    valid_end -= 1

                valid_entities.append([valid_start, valid_end, label])
            cleaned_data.append([text, {'entities': valid_entities}])
        return cleaned_data

In [11]:
docs = utils.v2Tov3Converter(train_data)

In [26]:
# initialize config.cfg file
!spacy init config --lang en --pipeline ner config.cfg --force

ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [27]:
# train the model using config.cfg file. and save the model in trained_model folder.
!python -m spacy train config.cfg --output ./trained_model/ --paths.train ./train.spacy --paths.dev ./train.spacy

ℹ No output directory provided
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2021-11-02 12:07:56,221] [INFO] Set up nlp object from config
[2021-11-02 12:07:56,248] [INFO] Pipeline: ['tok2vec', 'ner']
[2021-11-02 12:07:56,260] [INFO] Created vocabulary
[2021-11-02 12:07:56,277] [INFO] Finished initializing nlp object
[2021-11-02 12:08:04,595] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.8/runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/opt/anaconda3/lib/python3.8/runpy.py", line 87, in _run_code
    exec(code, run_g